In [1]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import pandas as pd
import geopandas as gpd
import textwrap
from geojson import Feature, FeatureCollection, Point

In [2]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [3]:
df = pd.read_csv('Miami Neighborhoods New Construction - Brickell.csv')

In [4]:
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
project_gdf = gpd.read_file('Brickell.kml', driver = 'KML')
project_gdf['center_point'] = project_gdf.geometry.centroid

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_57075/2050377854.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  project_gdf['center_point'] = project_gdf.geometry.centroid


In [5]:
project_gdf

,Name,Description,geometry,center_point
0,Brickell,,"POLYGON Z ((-80.19989 25.74976 0.00000, -80.19...",POINT (-80.19475 25.76096)


In [6]:
df

,address,developer,units,acerage,price_sale,price_loan,loan_financer,category,description,story_link,story_attribution
0,"888 Brickell Avenue, Miami FL","JDS Development Group, Major Food Group",259.0,NaN,na,na,na,proposed,"<a href=""https://therealdeal.com/miami/2022/05...",https://therealdeal.com/miami/2022/05/25/jds-m...,NaN
1,"102 Southwest Eighth Street, Miami FL",PMG,803.0,NaN,na,na,NaN,proposed,"<a href=""https://therealdeal.com/miami/2022/04...",https://therealdeal.com/miami/2022/04/20/pmg-p...,NaN
2,"609 Brickell Avenue, Miami FL",13th Floor Investments & Key International,NaN,2.2,na,na,NaN,proposed,"<a href=""https://therealdeal.com/miami/2022/02...",https://therealdeal.com/miami/2022/02/28/brick...,NaN
3,"90 Southwest Eighth Street, Miami, FL 33130",Gazit Globe Group,504.0,NaN,29500000,na,NaN,planned,"<a href=""https://therealdeal.com/miami/2022/02...",https://therealdeal.com/miami/2022/02/18/gazit...,NaN
4,"1420 South Miami Avenue, Miami, FL 33130",Mast Capital,1250.0,NaN,103000000,na,NaN,planned,"<a href=""https://therealdeal.com/miami/2021/12...",https://therealdeal.com/miami/2021/12/27/769192/,NaN
5,"99 Southwest Seventh Street, Miami, FL",Harvey Hernandez,NaN,1.6,50500000,55000000,NaN,planned,"<a href=""https://therealdeal.com/miami/2021/09...",https://therealdeal.com/miami/2021/09/15/harve...,NaN
6,"175 Southeast 25th Road, Miami FL",OKO Group,135.0,NaN,na,128300000,Bank OZK,under construction,"<a href=""https://therealdeal.com/miami/2021/04...",https://therealdeal.com/miami/2021/04/20/billi...,NaN
7,"1 Southside Park, Miami FL",JDS Development Group (Michael Stern),1000.0,1.5,23500000,na,NaN,under construction,"<a href=""https://therealdeal.com/miami/2021/04...",https://therealdeal.com/miami/2021/04/09/micha...,NaN
8,"444 Brickell Avenue, Miami FL","The Related Group, Baccarat",1400.0,4.0,104000000,na,NaN,planned,"<a href=""https://therealdeal.com/miami/2021/02...",https://therealdeal.com/miami/2021/02/04/relat...,NaN
9,"1809 Brickell Avenue, Miami, FL",Related Group & Integra Investments,354.0,3.0,14000000,na,NaN,planned,"<a href=""https://therealdeal.com/miami/2021/12...",https://therealdeal.com/miami/2021/12/16/relat...,NaN


In [7]:
df['loc'] = df['address'].apply(geolocator.geocode, timeout=10)

In [8]:
df['loc']

0     (888 Brickell Ave, Miami, FL 33131, USA, (25.7...
1     (102 SE 8th St, Miami, FL 33131, USA, (25.7664...
2     (609 Brickell Ave, Miami, FL 33131, USA, (25.7...
3     (90 SW 8th St, Miami, FL 33130, USA, (25.76606...
4     (1420 S Miami Ave, Miami, FL 33130, USA, (25.7...
5     (99 SW 7th St, Miami, FL 33130, USA, (25.76748...
6     (175 SE 25th Rd, Miami, FL 33129, USA, (25.749...
7     (1133 SW 2nd Ave, Miami, FL 33130, USA, (25.76...
8     (444 Brickell Ave, Miami, FL 33131, USA, (25.7...
9     (1809 Brickell Ave, Miami, FL 33129, USA, (25....
10    (1399 SW 1st Ave, Miami, FL 33130, USA, (25.76...
11    (239 SW 9th St, Miami, FL 33130, USA, (25.7655...
Name: loc, dtype: object

In [9]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

In [10]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

In [11]:
# df['image_img_test'] = ' '

In [12]:
# df.at[0,'image_img_test']=('<img src="Sean_Combs_2010.jpg" width="256" height="174">')
# df.at[1,'image_img_test']=('<img src="Sean_Combs_2010.jpg" width="256" height="174">')
# df.at[2,'image_img_test']=('<img src="DJ_Khaled_2012.jpg" width="256" height="188">')
# df.at[3,'image_img_test']=('<img src="LIL_WAYNE.jpg" width="256" height="252">')
# df.at[4,'image_img_test']=('<img src="Lil_Pump.jpg" width="256" height="190">')
# df.at[5,'image_img_test']=('<img src="Tripplie_Redd.jpg" width="256" height="170">')
# df.at[6,'image_img_test']=('<img src="Rick_Ross.jpg" width="256" height="199">')
# df.at[7,'image_img_test']=('<img src="Kodak_Black.jpg" width="256" height="136">')

In [13]:
df.columns

Index(['address', 'developer', 'units', 'acerage', 'price_sale', 'price_loan',
       'loan_financer', 'category', 'description', 'story_link',
       'story_attribution', 'loc', 'point', 'lat', 'lon', 'altitude'],
      dtype='object')

In [14]:
import folium
from folium.plugins import MarkerCluster
import textwrap

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=14.5)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'New Development in Brickell')

for i, r in project_gdf.iterrows():
    sim_geo = gpd.GeoSeries(r['geometry'])
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j)
    geo_j.add_to(m)


for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["address"], r["developer"], r["description"])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    italic_start = '<em>'
    italic_end = '</em>'
    text = f'Address: {bold_start}{info[0]}{bold_end}{new_line}\
    Developer(s): {bold_start}{italic_start}{info[1]}{italic_end}{bold_end}{new_line}\
    {bold_start}{info[2]}{bold_end}{new_line}'
#     text = text.replace(". ",".<br>")
    popup = folium.Popup(text, max_width=300)
    folium.Marker(location=location,
                      popup=popup)\
    .add_to(m)

m.get_root().html.add_child(folium.Element(title_html))

m

In [15]:
m.save('index.html')